In [27]:
from datasets import load_dataset
import pandas as pd
import os

#dataset = load_dataset("maharshipandya/spotify-tracks-dataset")

In [28]:
lyrics1_path = 'lyrics1_dataset.csv'
lyrics2_folder_path = 'lyrics2_dataset'

no_lyrics_text = "lyrics for this song have yet to be released please check back once the song has been released"

# Read the first dataset
df_master = pd.read_csv(lyrics1_path, usecols=['Artist Name', 'Song Name', 'Lyrics'])
#df_master['Lyrics'] = df_temp['Lyrics'].str.replace(',', '')
df_master.columns = ['Artist', 'Title', 'Lyrics']  # Standardize column names

# Iterate through each CSV file in the lyrics2_dataset folder
for file in os.listdir(lyrics2_folder_path):
    file_path = os.path.join(lyrics2_folder_path, file)
    if os.path.isfile(file_path) and file_path.endswith('.csv'):
        # Read the dataset
        df_temp = pd.read_csv(file_path, usecols=['Artist', 'Title', 'Lyric'])

        # Filter out rows with no lyrics
        df_temp = df_temp.loc[~df_temp['Lyric'].str.contains(no_lyrics_text, case=False, na=False)]
        
        df_temp.columns = ['Artist', 'Title', 'Lyrics']  # Standardize column names

        # Concatenate with the master DataFrame
        df_master = pd.concat([df_master, df_temp], ignore_index=True)

        # Remove commas from lyrics
        df_master['Lyrics'] = df_master['Lyrics'].str.replace(',', '')

output_path = 'combined_lyrics_dataset.csv'
df_master.to_csv(output_path, index=False)

print("Dataset combined and saved.")

Dataset combined and saved.


In [8]:
features_df = pd.read_csv('features_dataset.csv')
combined_lyrics_df = df_master
features_df['artists'] = features_df['artists'].str.split(';')
features_exploded_df = features_df.explode('artists')

#merged_df = pd.merge(combined_lyrics_df, features_exploded_df, left_on=['Artist', 'Title'], right_on=['artists', 'track_name'])
merged_df = pd.merge(combined_lyrics_df, features_exploded_df, left_on=['Title'], right_on=['track_name'])

# Selecting columns for the final merged dataset
final_columns = ['Artist', 'Title', 'Lyrics', 'track_id', 'popularity', 'duration_ms', 'explicit', 
                 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']
final_merged_df = merged_df[final_columns]

final_merged_df.to_csv('final_lyrics_features_combined.csv', index=False)


In [9]:
## this is not working bruh

final_merged_df = final_merged_df.drop_duplicates(subset=['Artist', 'Title'], keep='first')
final_merged_df.to_csv('final_lyrics_features_combined.csv', index=False)